In [ ]:
import requests
import csv
import re
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
CHROMA_PATH = "chroma"
url = 'https://7jb5pn2q0l4lf2-8000.proxy.runpod.net/v1/completions'

PROMPT_TEMPLATE = """
Answer and explain the question with Thai language based only on the following context:
โดยทุจริต = เพื่อแสวงหาประโยชน์ที่มิควรได้โดยชอบด้วยกฎหมายสําหรับตนเองหรือผู้อื่น
ทางสาธารณะ = ทางบกหรือทางน้ําสําหรับประชาชนใช้ในการจราจร และให้หมายความรวมถึงทางรถไฟและทางรถรางที่มีรถเดิน สําหรับประชาชนโดยสารด้วย
สาธารณสถาน = สถานที่ใด ๆ ซึ่งประชาชนมีความชอบธรรมที่จะเข้าไปได้
เคหสถาน = ที่ซึ่งใช้เป็นที่อยู่อาศัย เช่น เรือน โรง เรือ หรือแพซึ่งคนอยู่อาศัยและให้หมายความรวมถึงบริเวณของที่ซึ่งใช้เป็นที่อยู่อาศัยนั้นด้วย จะมีรั้วล้อมหรือไม่ก็ตาม
อาวุธ = หมายความรวมถึงสิ่งซึ่งไม่เป็นอาวุธโดยสภาพ แต่ซึ่งได้ใช้หรือเจตนาจะใช้ประทุษร้ายร่างกายถึงอันตรายสาหัสอย่างอาวุธ
ใช้กําลังประทุษร้าย = ทําการประทุษร้ายแก่กายหรือจิตใจของบุคคล ไม่ว่าจะทําด้วยใช้แรงกายภาพหรือด้วยวิธีอื่นใด และให้หมายความรวมถึงการกระทําใด ๆ ซึ่งเป็นเหตุให้บุคคลหนึ่งบุคคลใดอยู่ในภาวะที่ไม่สามารถขัดขืนได้ 
ไม่ว่าจะโดยใช้ยาทําให้มึนเมา สะกดจิต หรือใช้วิธีอื่นใดอันคล้ายคลึงกัน
เอกสาร = กระดาษหรือวัตถุอื่นใดซึ่งได้ทําให้ปรากฏความหมายด้วยตัวอักษร ตัวเลข ผัง หรือแผนแบบอย่างอื่น จะเป็นโดยวิธีพิมพ์ ถ่ายภาพ หรือวิธีอื่นอันเป็นหลักฐานแห่งความหมายนั้น
เอกสารราชการ = เอกสารซึ่งเจ้าพนักงานได้ทําขึ้นหรือรับรองในหน้าที่ และให้หมายความรวมถึงสําเนาเอกสารนั้น ๆ ที่เจ้าพนักงานได้รับรองในหน้าที่ด้วย
เอกสารสิทธิ = เอกสารที่เป็นหลักฐานแห่งการก่อ เปลี่ยนแปลงโอน สงวนหรือระงับซึ่งสิทธิ
ลายมือชื่อ = หมายความรวมถึงลายพิมพ์นิ้วมือและเครื่องหมายซึ่งบุคคลลงไว้แทนลายมือชื่อของตน
กลางคืน = เวลาระหว่างพระอาทิตย์ตกและพระอาทิตย์ขึ้น
คุมขัง = คุมตัว ควบคุม ขัง กักขังหรือจําคุก
ค่าไถ่ = ทรัพย์สินหรือประโยชน์ที่เรียกเอา หรือให้เพื่อแลกเปลี่ยนเสรีภาพของผู้ถูกเอาตัวไป ผู้ถูกหน่วงเหนี่ยวหรือผู้ถูกกักขัง
บัตรอิเล็กทรอนิกส์ หมายถึง 
(ก)เอกสารหรือวัตถุอื่นใดไม่ว่าจะมีรูปลักษณะใดที่ผู้ออกได้ออกให้แก่ผู้มีสิทธิใช้ ซึ่งจะระบุชื่อหรือไม่ก็ตาม โดยบันทึกข้อมูลหรือรหัสไว้ด้วยการประยุกต์ใช้วิธีการทางอิเล็กตรอนไฟฟ้า คลื่นแม่เหล็กไฟฟ้า 
หรือวิธีอื่นใดในลักษณะคล้ายกัน ซึ่งรวมถึงการประยุกต์ใช้วิธีการทางแสงหรือวิธีการทางแม่เหล็กให้ปรากฏความหมายด้วยตัวอักษร ตัวเลข รหัส หมายเลขบัตร หรือสัญลักษณ์อื่นใด ทั้งที่สามารถมองเห็นและมองไม่เห็นด้วยตาเปล่า
(ข)ข้อมูล รหัส หมายเลขบัญชี หมายเลขชุดทางอิเล็กทรอนิกส์หรือเครื่องมือทางตัวเลขใด ๆ ที่ผู้ออกได้ออกให้แก่ผู้มีสิทธิใช้ โดยมิได้มีการออกเอกสารหรือวัตถุอื่นใดให้ แต่มีวิธีการใช้ในทํานองเดียวกับ (ก) หรือ
(ค)สิ่งอื่นใดที่ใช้ประกอบกับข้อมูลอิเล็กทรอนิกส์เพื่อแสดงความสัมพันธ์ระหว่างบุคคลกับข้อมูลอิเล็กทรอนิกส์ โดยมีวัตถุประสงค์เพื่อระบุตัวบุคคลผู้เป็นเจ้าของ
หนังสือเดินทาง = เอกสารสําคัญประจําตัวไม่ว่าจะมีรูปลักษณะใดที่รัฐบาลไทย รัฐบาลต่างประเทศ หรือองค์การระหว่างประเทศออกให้แก่บุคคลใด เพื่อใช้แสดงตนในการเดินทางระหว่างประเทศ 
และให้หมายความรวมถึงเอกสารใช้แทนหนังสือเดินทางและแบบหนังสือเดินทางที่ยังไม่ได้กรอกข้อความเกี่ยวกับผู้ถือหนังสือเดินทางด้วย
เจ้าพนักงาน = บุคคลซึ่งกฎหมายบัญญัติว่าเป็นเจ้าพนักงานหรือได้รับแต่งตั้งตามกฎหมายให้ปฏิบัติหน้าที่ราชการ ไม่ว่าเป็นประจําหรือครั้งคราว และไม่ว่าจะได้รับค่าตอบแทนหรือไม่
สื่อลามกอนาจารเด็ก = วัตถุหรือสิ่งที่แสดงให้รู้หรือเห็นถึงการกระทําทางเพศของเด็กหรือกับเด็กซึ่งมีอายุไม่เกินสิบแปดปี โดยรูป เรื่อง หรือลักษณะสามารถสื่อไปในทางลามกอนาจาร ไม่ว่าจะอยู่ในรูปแบบของเอกสาร ภาพเขียน ภาพพิมพ์ ภาพระบายสี สิ่งพิมพ์
รูปภาพ ภาพโฆษณา เครื่องหมาย รูปถ่าย ภาพยนตร์ แถบบันทึกเสียง แถบบันทึกภาพ หรือรูปแบบอื่นใดในลักษณะทํานองเดียวกัน และให้หมายความรวมถึงวัตถุหรือสิ่งต่าง ๆ ข้างต้นที่จัดเก็บในระบบคอมพิวเตอร์หรือในอุปกรณ์อิเล็กทรอนิกส์อื่นที่สามารถแสดงผลให้เข้าใจความหมายได้

{prompt}

{context}

---

Answer the question based on the above context : {question}
"""

In [ ]:
def arabic_to_thai_number(input_str: str) -> str:
    thai_digits = str.maketrans("0123456789", "๐๑๒๓๔๕๖๗๘๙")
    return input_str.translate(thai_digits)

In [ ]:
import json

data = None
mapping_prompt = None
with open('mapping_chunk.json', 'r') as file:
    data = json.load(file)

for value in data:
    print(data[value][0], data[value][1])

with open('mapping_prompt.json', 'r',encoding="utf-8") as file:
    mapping_prompt = json.load(file)

for value in mapping_prompt:
    print(f"{value} : {mapping_prompt[value]}")

In [ ]:
def query_vllm(prompt: str):
    payload = {
      "model": "openthaigpt/openthaigpt1.5-7b-instruct",
      "prompt": prompt,
      "max_tokens": 200,
      "temperature": 0.7,
      "stream": False
    }

    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json'
    }
    
    response = requests.post(url, headers=headers, json=payload)
    
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code} - {response.text}"

In [ ]:
def response(question):
    query_text = question
    query_text = arabic_to_thai_number(query_text)
    embedding_function = HuggingFaceEmbeddings(
        model_name="intfloat/multilingual-e5-large",
        model_kwargs={"device": "cpu"},
        encode_kwargs={"normalize_embeddings": True},
    )
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    results = db.similarity_search_with_relevance_scores(query_text, k=5)
    print(f"Query: {query_text}")
    print(f"Results: {results}")
    
    if len(results) == 0 or results[0][1] < 0.75:
        print(f"Unable to find matching results.")
        return

    files = set()
    prompt_texts = []
    context_texts = []
    for doc, score in results:
        chunk_number = doc.metadata["chunk_number"]
        # print(f"chunk_number : {chunk_number}")
        for value in data:
            if chunk_number >= data[value][0] and chunk_number <= data[value][1]:
                files.add(value)
                break
            else:
                continue

        context_texts.append(f"{doc.page_content}")

    for value in files:
        prompt_texts.append(f"{mapping_prompt[value]}")        

    context_text = "".join(context_texts)

    prompt_text = "".join(prompt_texts)
    
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=question, prompt=prompt_text)
    print(f"\n\n\n\nPrompt: {prompt}")

    response_text = query_vllm(prompt)
    print(f"Response Text: {response_text}")

    if isinstance(response_text, dict) and 'choices' in response_text:
        return response_text['choices'][0]['text']
    else:
        return response_text  # if it's already a string


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from langchain.embeddings import HuggingFaceEmbeddings
import numpy as np

def calculate_cosine_similarity(generated_answers, reference_answers):
    """
    Calculates cosine similarity between the generated and reference answers.

    Args:
        generated_answers (list[str]): Generated answers.
        reference_answers (list[str]): Reference (gold standard) answers.

    Returns:
        float: Average cosine similarity between the answers.
    """
    # Initialize the HuggingFace embedding model
    embedding_model = HuggingFaceEmbeddings(
        model_name="intfloat/multilingual-e5-large",
        model_kwargs={"device": "cpu"},
        encode_kwargs={"normalize_embeddings": True},
    )
    
    # Use embed_documents to generate embeddings for lists of documents
    generated_embeddings = embedding_model.embed_documents(generated_answers)
    reference_embeddings = embedding_model.embed_documents(reference_answers)
    
    # Convert to numpy arrays for compatibility
    generated_embeddings = np.array(generated_embeddings)
    reference_embeddings = np.array(reference_embeddings)
    
    # Calculate cosine similarity for each pair
    similarities = []
    for gen_emb, ref_emb in zip(generated_embeddings, reference_embeddings):
        similarity = cosine_similarity([gen_emb], [ref_emb])[0][0]
        similarities.append(similarity)
    
    # Return the average similarity score
    avg_similarity = np.mean(similarities)
    
    return avg_similarity


In [ ]:
question_pattern = r"ข้อใด.*?(?=\s*ก\.)"
LABELS_FILE = 'output.csv'

with open(LABELS_FILE, 'r', newline='', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    dataset = [row for row in reader]

gen_answer = []
real_answer = []

In [ ]:
for row in dataset:
    query = row['Question']  # Assuming the CSV has a 'Question' column
    ans = row['Answer']
    match = re.search(question_pattern, query, re.DOTALL)
    if match :
        question = match.group()
    else :
        question = query
    print(question)
    
    response_text = response(question)
    gen_answer.append(response_text)
    real_answer.append(ans)
    print(f"Qusetion : {query}")
    print(f"LLM Answer : {response_text}")
    print(f"Real Answer : {ans}")

In [ ]:
similarity = calculate_cosine_similarity(gen_answer, real_answer)
print(similarity) 

In [ ]:
print(len(gen_answer))
for i in range(len(gen_answer)):
    print(f"{i}")
    print(f"Gen answer : {gen_answer[i]}")
    print(f"Real answer : {real_answer[i]}")
    print()

In [ ]:
response("การลักทรัพย์มูลค่าไม่เกิน 500 บาท มีความผิดฐานอะไร")

In [ ]:
def query_vllm(prompt: str):
    payload = {
        "model": "scb10x/llama3.1-typhoon2-8b-instruct",
        "prompt": prompt,
        "max_tokens": 450,
        "temperature": 0.7,
        "top_p": 0.9,
        "frequency_penalty": 0.4,
        "presence_penalty": 0.4,
        "stream": False
    }


    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json'
    }
    
    response = requests.post(url, headers=headers, json=payload)
    
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code} - {response.text}"

In [ ]:
import csv
import pandas as pd
import matplotlib.pyplot as plt

# Read questions and answers
with open("questions.txt", "r", encoding="utf-8") as q_file:
    questions = [line.strip() for line in q_file.readlines()]

with open("answers.txt", "r", encoding="utf-8") as a_file:
    answers = [line.strip() for line in a_file.readlines()]

# Ensure both files have the same number of lines
if len(questions) != len(answers):
    raise ValueError("Mismatch between number of questions and answers")

# Save to CSV file
csv_filename = "Question.csv"
with open(csv_filename, "w", encoding="utf-8", newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Question", "Answer"])
    writer.writerows(zip(questions, answers))

print("Data successfully saved to", csv_filename)

# Load and visualize data
df = pd.read_csv(csv_filename)
print(df.head())  # Display first few rows